In [1]:
import sys
sys.path.insert(1, '../')

In [2]:
import helpers.preprocesamiento as procs
import spacy
import pickle
import numpy as np

In [4]:
with open("pickles/lectura_sample.pickle", "rb") as f:
    df, f_ini, f_fin = pickle.load(f) 

In [7]:
len(df.screen_name.unique())

112

In [8]:
len(df)

33600

# TODO: Revisar acentos

In [4]:
df_char_r = procs.remove_chars(df)
nlp = spacy.load('es_core_news_lg')
stop_words = ["y", "a", "ante", "a", "bajo", "con", "de", "desde", "durante", "en", "entre", "excepto", "hacia",
              "hasta", "mediante", "para", "por", "salvo", "según", "sin", "sobre", "tras", "y", "e", "ni", "o", "u",
              "que", "si", "como", "donde", "quien", "cual", "cuyo", "cuanto", "el", "lalos", "las"]
for word in stop_words:
    nlp.vocab[word].is_stop = True
docs = list(df_char_r['text_clean'])
docs = procs.entidades(docs, nlp)
docs = procs.bigramas(docs)
dictionary = procs.filtrar_extremos(docs)

In [5]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

Borramos tweets que:
- Tengan menos de dos palabras

In [6]:
id_borrar =[i for i in range(0, len(corpus)) if len(corpus[i]) <= 2]

# Split Test Train

Se repocesan de nuevo todo para:
1. Separamos datos en prueba y entrenamiento.

In [9]:
df_tweets_vacios = (df_char_r
                    .drop(index = id_borrar)
                    .reset_index(drop=True)
                   )

Lo separamos a nivel autor

In [10]:
sample = (df_tweets_vacios
          .assign(id_ = np.arange(len(df_tweets_vacios)))
          .groupby("screen_name")
          .apply(lambda df: df.sample(frac=0.8, random_state=811))
         )

In [11]:
train_set = df_tweets_vacios.copy().iloc[sample["id_"],]
test_set = df_tweets_vacios.copy().drop(index =train_set.index)

2. Eliminar del corpus tweets vacios.

In [12]:
docs_tweets_vacios = list(df_tweets_vacios['text_clean'])
docs_tweets_vacios = procs.entidades(docs_tweets_vacios, nlp)
docs_tweets_vacios = procs.bigramas(docs_tweets_vacios)

In [13]:
dictionary_docs_tweets_vacios = procs.filtrar_extremos(docs_tweets_vacios)

In [14]:
corpus_train = [dictionary_docs_tweets_vacios.doc2bow(docs_tweets_vacios[doc_id]) for doc_id in train_set.index]
corpus_test = [dictionary_docs_tweets_vacios.doc2bow(docs_tweets_vacios[doc_id]) for doc_id in test_set.index]

In [15]:
author2doc_train = {aut: [] for aut in train_set["screen_name"].unique()}

for index, row in train_set.reset_index(drop=True).iterrows():
    author2doc_train[row['screen_name']].append(index)

In [16]:
author2doc_test = {aut: [] for aut in test_set["screen_name"].unique()}

for index, row in test_set.reset_index(drop=True).iterrows():
    author2doc_test[row['screen_name']].append(index)

# Guardar

In [17]:
with open("pickles/preprocesamiento.pickle", "wb") as f:
    pickle.dump((
        dictionary_docs_tweets_vacios,
        corpus_train,
        corpus_test,
        author2doc_train,
        author2doc_test,
        train_set,
        test_set
    ) , f)